# Analyse [ontology-mapping-chatbot](https://github.com/opendatasoft/ontology-mapping-chatbot)

## Preparations

After cloning the [project](https://github.com/opendatasoft/ontology-mapping-chatbot) in a local directory and doing the necessary installation following the `README` of the project we can run the application at http://127.0.0.1:8000/.
(Change `DEBUG = True` in `chatbot_app/settings.py` if it is complaining.) We are then able to import the `roman-emperors@public` and do the mapping in the UI.

In order to be able to call modules and examin individual aspects of the app we copy the complete project to the homedirectory of our notebook.

In [4]:
!cp -R ../ontology-mapping-chatbot/ .

In [6]:
!ls -l

total 56
-rw-r--r--   1 ecco  staff  1069 Mar  9 12:26 LICENSE
-rw-r--r--   1 ecco  staff  7452 Mar  9 12:26 README.md
drwxr-xr-x  12 ecco  staff   384 Mar  9 12:26 chat
drwxr-xr-x   6 ecco  staff   192 Mar  9 12:26 chatbot
drwxr-xr-x   9 ecco  staff   288 Mar  9 12:27 chatbot_app
drwxr-xr-x   6 ecco  staff   192 Mar  9 12:26 data_dumps
-rw-r--r--   1 ecco  staff  1364 Mar  9 12:40 explore.ipynb
drwxr-xr-x   4 ecco  staff   128 Mar  9 12:26 img
-rwxr-xr-x   1 ecco  staff   373 Mar  9 12:26 install.sh
drwxr-xr-x   4 ecco  staff   128 Mar  9 12:26 logs
-rwxr-xr-x   1 ecco  staff   809 Mar  9 12:26 manage.py
-rw-r--r--   1 ecco  staff   179 Mar  9 12:26 requirements.txt
drwxr-xr-x   4 ecco  staff   128 Mar  9 12:26 results
drwxr-xr-x   4 ecco  staff   128 Mar  9 12:26 tests
drwxr-xr-x  15 ecco  staff   480 Mar  9 12:26 utils


We also have the datadumps which were put there when we ran `install.sh`.

In [10]:
ls -l data_dumps/dbpedia/en

total 527944
-rw-r--r--  1 ecco  staff  162329691 Mar  9 12:26 instance_type.hdt
-rw-r--r--  1 ecco  staff  107974152 Mar  9 12:26 instance_type.hdt.index.v1-1


We can now import individual modules in our notebook environment.

The modules use settings in the `chatbot_app/settings`. We can mimic the django configuration by setting an environment variable.

In [165]:
import os
import json
from IPython.display import HTML
from IPython.display import JSON

from hdt import HDTDocument
from django.utils.encoding import smart_str
from collections import Counter

# imports from ontology-mapping-chatbot
import chatbot_app.settings as settings
import utils.ods_catalog_api as ODSCatalogApi
import utils.ods_dataset_api as ODSDatasetApi
import chatbot.semantic_engine as SemanticEngine
import utils.dbpedia_ner as DBPediaNER
import utils.lov_ods_api as LovApi

# set the environment variable
os.environ['DJANGO_SETTINGS_MODULE'] = 'chatbot_app.settings'

## Get data
From the api-handler in `chat.views.correspondance_api.get_correspondances` and its dependencies we learn...
```python
@require_http_methods(['GET'])
def get_correspondances(request, dataset_id):
    logging.getLogger("results_logger").info("[{}] Starting semantization".format(dataset_id))
    ods_dataset_metas = ODSCatalogApi.dataset_meta_request(dataset_id)
    ods_dataset_records = ODSDatasetApi.dataset_records_request(dataset_id, settings.RECORD_NUMBER)['records']
    correspondances = SemanticEngine.init_correspondances_set(ods_dataset_metas, ods_dataset_records)
    if not correspondances.get('classes'):
        logging.getLogger("results_logger").info("[{}] No correspondances found".format(dataset_id))
    logging.getLogger("results_logger").info("[{}] Starting semantization".format(dataset_id))
    response = HttpResponse(
        json.dumps(correspondances),
        content_type='application/json')
    response['Access-Control-Allow-Origin'] = '*'
    return response
```
- ... that a dataset metadata can be obtained from https://data.opendatasoft.com/api/datasets/1.0/roman-emperors@public. 
- ... that an amount of records from that dataset can be obtained from https://data.opendatasoft.com/api/records/1.0/search/?dataset=roman-emperors@public&rows=50

Lets download our first input-data.

In [46]:
dataset_id = 'roman-emperors@public'

### Metadata on the dataset

In [15]:
ODSCatalogApi.dataset_meta_request??

Signature: ODSCatalogApi.dataset_meta_request(dataset_id)
Docstring: <no docstring>
Source:   
def dataset_meta_request(dataset_id):
    if dataset_id:
        params = {'apikey': settings.DATA_API_KEY}
        request = requests.get(DATA_CATALOG_API_URL.format(dataset_id), params, timeout=Requester.get_timeout(), headers=Requester.create_ods_headers())
        request.raise_for_status()
        return request.json()
    else:
        raise DatasetIdMissing
File:      ~/git/ontmap/utils/ods_catalog_api.py
Type:      function


In [33]:
ods_dataset_metas = ODSCatalogApi.dataset_meta_request(dataset_id)
type(ods_dataset_metas)

dict

In [43]:
HTML('<a href="{}" target="_blank">license of the dataset</a>'.format(ods_dataset_metas['metas']['license']))

Before we do a copyright infringement!! :<br/>
All fragments showing portions of the dataset *roman-emperors@public*: <span style="color: red;">**Copyright (c) 2016 Zoni Nation**</span>

In [44]:
ods_dataset_metas['metas']['keyword'] # etc.

['Roman', 'Empire', 'Emperor', 'Reign', 'History']

See full metadata: https://data.opendatasoft.com/api/datasets/1.0/roman-emperors@public

### Get the dataset

In [51]:
ODSDatasetApi.dataset_records_request??

Signature: ODSDatasetApi.dataset_records_request(dataset_id, rows=10)
Docstring: <no docstring>
Source:   
def dataset_records_request(dataset_id, rows=10):
    if dataset_id:
        params = {'dataset': dataset_id, 'rows': rows, 'apikey': settings.DATA_API_KEY}
        request = requests.get(DATA_RECORD_API_URL, params, timeout=Requester.get_timeout(), headers=Requester.create_ods_headers())
        request.raise_for_status()
        return request.json()
    else:
        raise DatasetIdMissing
File:      ~/git/ontmap/utils/ods_dataset_api.py
Type:      function


In [49]:
settings.RECORD_NUMBER

50

In [50]:
ods_dataset_records = ODSDatasetApi.dataset_records_request(dataset_id, settings.RECORD_NUMBER)['records']
type(ods_dataset_records)

list

In [53]:
len(ods_dataset_records)

50

In [166]:
JSON(ods_dataset_records[0])

<IPython.core.display.JSON object>

In [58]:
print('The death cause of emperor {} was {}.'.format(ods_dataset_records[0]['fields']['name_full'], ods_dataset_records[0]['fields']['cause']))

The death cause of emperor TIBERIVS CAESAR DIVI AVGVSTI FILIVS AVGVSTVS was Assassination.


See full record set: https://data.opendatasoft.com/api/records/1.0/search/?dataset=roman-emperors@public&rows=50

## Get correspondances

In [60]:
SemanticEngine.init_correspondances_set??

Signature:
SemanticEngine.init_correspondances_set(
    ['ods_dataset_metas', 'ods_dataset_records'],
)
Docstring: <no docstring>
Source:   
def init_correspondances_set(ods_dataset_metas, ods_dataset_records):
    language = get_dataset_language(ods_dataset_metas)
    candidate_correspondances = {'classes': get_dataset_classes(ods_dataset_records, ods_dataset_metas, language),
                                 'properties': get_dataset_properties(ods_dataset_metas)}
    return candidate_correspondances
File:      ~/git/ontmap/chatbot/semantic_engine.py
Type:      function


In [61]:
correspondances = SemanticEngine.init_correspondances_set(ods_dataset_metas, ods_dataset_records)

In [62]:
type(correspondances)

dict

In [65]:
correspondances

{'classes': [{'uri': 'http://schema.org/Place',
   'class': 'Place',
   'description': b'Place',
   'sub': ['http://rdfs.co/juso/SpatialThing', 'http://schema.org/Thing'],
   'eq': ['http://purl.org/goodrelations/v1#Location'],
   'label': 'Birth City',
   'field_name': 'birth_cty'},
  {'uri': 'http://schema.org/Person',
   'class': 'Person',
   'description': b'A person.',
   'sub': ['http://purl.org/dc/terms/Agent',
    'http://purl.org/goodrelations/v1#BusinessEntity',
    'http://rhizomik.net/ontologies/copyrightonto.owl#LegalPerson',
    'http://schema.org/Thing',
    'http://www.w3.org/2000/10/swap/pim/contact#Person',
    'http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing',
    'http://xmlns.com/foaf/0.1/Agent',
    'http://xmlns.com/foaf/0.1/Person'],
   'eq': ['http://www.w3.org/2000/10/swap/pim/contact#Person',
    'http://xmlns.com/foaf/0.1/Person'],
   'label': 'Name',
   'field_name': 'name'},
  {'uri': 'http://schema.org/Place',
   'class': 'Place',
   'description': b

`correspondances` contains 2 dicts: *classes* and *properties*. So, where is this comming from?

### Correspondance classes

Classes from the `correspondances` dict are looked up by querying the HDT-documents in `data_dumps`.

In [66]:
SemanticEngine.get_dataset_language??

Signature: SemanticEngine.get_dataset_language(ods_dataset_metas)
Docstring: <no docstring>
Source:   
def get_dataset_language(ods_dataset_metas):
    if 'metas' in ods_dataset_metas:
        if 'language' in ods_dataset_metas['metas']:
            return ods_dataset_metas['metas']['language']
    return 'eng'
File:      ~/git/ontmap/chatbot/semantic_engine.py
Type:      function


It looks up the language in the metadata, otherwise assumes it is 'eng'.

In [67]:
SemanticEngine.get_dataset_classes??

Signature:
SemanticEngine.get_dataset_classes(
    ['ods_dataset_records', 'ods_dataset_metas', "language='en'"],
)
Docstring: <no docstring>
Source:   
def get_dataset_classes(ods_dataset_records, ods_dataset_metas, language='en'):
    candidates_classes = {}
    # Search classes using Named Entity Recognition on instances
    for record in ods_dataset_records:
        for field, value in record['fields'].items():
            if hasNoNumbers(value):
                types = DBPediaNER.entity_types_request(value, language)
                if not types:
                    # DBPedia could not find any class for this field
                    types = YagoNER.entity_types_request(value, language)
                if types:
                    if field in candidates_classes:
                        candidates_classes[field].extend(types)
                    else:
                        candidates_classes[field] = types
    correspondances = []
    for field, classes in candidates_classes.it

For each *record* in `ods_dataset_records`, for each *field* and *value* in *record* `fields`, if *value* has no numbers, look up in DBPedia or Yago.<bf/>
Let's do that:

In [71]:
DBPediaNER.entity_types_request??

Signature: DBPediaNER.entity_types_request(query, language='en')
Docstring: <no docstring>
Source:   
def entity_types_request(query, language='en'):
    if query:
        query = query.encode("utf-8")
        query = to_dbpedia_format(query)
        if language == 'fr':
            subject_query_fr = FR_DBPEDIA_RESOURCE_URI.format(resource_name=query)
            (triples, cardinality) = fr_dbpedia.search_triples(subject_query_fr, "", "")
        else:
            subject_query = DBPEDIA_RESOURCE_URI.format(resource_name=query)
            (triples, cardinality) = eng_dbpedia.search_triples(subject_query, "", "")
        classes = []
        for triple in triples:
            cl = get_uri_suffix(triple[2])
            if not is_ignored(cl):
                classes.append(cl)
        if classes:
            return classes
    return None
File:      ~/git/ontmap/utils/dbpedia_ner.py
Type:      function


In [76]:
query = 'Rome'
language = 'en'
types = DBPediaNER.entity_types_request(value, language)
types

['Location', 'Place', 'PopulatedPlace', 'Place']

#### Correspondance classes from a method ...

In [124]:
DBPEDIA_RESOURCE_URI = "http://dbpedia.org/resource/{resource_name}"
eng_dbpedia = HDTDocument("data_dumps/dbpedia/en/instance_type.hdt")

def get_classes(value):
    query = value.encode("utf-8")
    query = query.decode().title()
    query = query.replace(' ', '_')
    subject_query = DBPEDIA_RESOURCE_URI.format(resource_name=query)
    print('subject_query="{}"'.format(subject_query))
    return [x for x in eng_dbpedia.search_triples(subject_query, "", "")[0]]

In [125]:
get_classes('Rome')

subject_query="http://dbpedia.org/resource/Rome"


[('http://dbpedia.org/resource/Rome',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Location'),
 ('http://dbpedia.org/resource/Rome',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Place'),
 ('http://dbpedia.org/resource/Rome',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/PopulatedPlace'),
 ('http://dbpedia.org/resource/Rome',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://schema.org/Place'),
 ('http://dbpedia.org/resource/Rome',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.w3.org/2002/07/owl#Thing'),
 ('http://dbpedia.org/resource/Rome',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.wikidata.org/entity/Q486972')]

In [127]:
get_classes('The Hague')

subject_query="http://dbpedia.org/resource/The_Hague"


[('http://dbpedia.org/resource/The_Hague',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Location'),
 ('http://dbpedia.org/resource/The_Hague',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Place'),
 ('http://dbpedia.org/resource/The_Hague',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/PopulatedPlace'),
 ('http://dbpedia.org/resource/The_Hague',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Settlement'),
 ('http://dbpedia.org/resource/The_Hague',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://schema.org/City'),
 ('http://dbpedia.org/resource/The_Hague',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://schema.org/Place'),
 ('http://dbpedia.org/resource/The_Hague',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.w3.org/2002/07/owl#Thing'),
 ('http://dbpedia.org/resource/The_Hague',
  'http

In [110]:
eng_dbpedia

<HDTDocument data_dumps/dbpedia/en/instance_type.hdt (~31254270 RDF triples)>

In [138]:
get_classes('arthur')

subject_query="http://dbpedia.org/resource/Arthur"


[('http://dbpedia.org/resource/Arthur',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Name'),
 ('http://dbpedia.org/resource/Arthur',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.w3.org/2002/07/owl#Thing'),
 ('http://dbpedia.org/resource/Arthur',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.wikidata.org/entity/Q202444'),
 ('http://dbpedia.org/resource/Arthur',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.wikidata.org/entity/Q82799')]

In [150]:
get_classes('George Washington')

subject_query="http://dbpedia.org/resource/George_Washington"


[('http://dbpedia.org/resource/George_Washington',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Agent'),
 ('http://dbpedia.org/resource/George_Washington',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://dbpedia.org/ontology/Person'),
 ('http://dbpedia.org/resource/George_Washington',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://schema.org/Person'),
 ('http://dbpedia.org/resource/George_Washington',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#Agent'),
 ('http://dbpedia.org/resource/George_Washington',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson'),
 ('http://dbpedia.org/resource/George_Washington',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.w3.org/2002/07/owl#Thing'),
 ('http://dbpedia.org/resource/George_Washington',
  'http://www.w3.org/1999/02/22

So that's entity recognition and further on in `SemanticEngine.get_dataset_classes` choose most common class etc.

### Class corespondances

In [151]:
SemanticEngine.get_class_correspondance??

Signature: SemanticEngine.get_class_correspondance(clss, language='en')
Docstring: <no docstring>
Source:   
def get_class_correspondance(clss, language='en'):
    response = {'uri': '', 'class': clss, 'description': clss, 'sub': [], 'eq': []}
    lov_results = LovApi.term_request(clss, term_type='class', language=language)["records"]
    for lov_result in lov_results:
        lov_result = lov_result['record']
        if is_valid(lov_result):
            response['uri'] = lov_result['fields']['uri']
            if lov_result['fields']['description'] and len(lov_result['fields']['description']) < 40:
                cleaned_description = BeautifulSoup(lov_result['fields']['description'], "html5lib").get_text().encode('utf8')
                response['description'] = cleaned_description
            elif lov_result['fields']['label']:
                cleaned_description = BeautifulSoup(lov_result['fields']['label'], "html5lib").get_text().encode('utf8')
                response['descripti

In [160]:
classes = get_classes('Rome')
common_class = Counter(classes).most_common(1)[0][0]
common_class = smart_str(common_class)
print('common_class =', common_class)
class_correspondance = SemanticEngine.get_class_correspondance(common_class)
class_correspondance

subject_query="http://dbpedia.org/resource/Rome"
common_class = ('http://dbpedia.org/resource/Rome', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://dbpedia.org/ontology/Location')


In [163]:
lov_result = LovApi.term_request(common_class)

See: http://lov.okfn.org/dataset/lov/api/v2/term/search?q=('http%3A%2F%2Fdbpedia.org%2Fresource%2FRome'%2C%20'http%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23type'%2C%20'http%3A%2F%2Fdbpedia.org%2Fontology%2FLocation')&type=class

In [164]:
lov_result

{'total_count': 26,
 'links': [{'href': 'https://data.opendatasoft.com/api/v2/catalog/datasets/linked-open-vocabularies-classes@public/records?start=0&rows=50&where=%28uri_suffix+like+%27%28http%3A%2F%2Fdbpedia.org%2Fresource%2F%27+OR+uri_suffix+like+%27rome%2C%27+OR+uri_suffix+like+%27http%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22%27+OR+uri_suffix+like+%27rdf%27+OR+uri_suffix+like+%27syntax%27+OR+uri_suffix+like+%27ns%23type%2C%27+OR+uri_suffix+like+%27http%3A%2F%2Fdbpedia.org%2Fontology%2F%27+OR+uri_suffix+like+%27location%29%27+OR+equivalent_classes_suffix+like+%27%28http%3A%2F%2Fdbpedia.org%2Fresource%2F%27+OR+equivalent_classes_suffix+like+%27rome%2C%27+OR+equivalent_classes_suffix+like+%27http%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22%27+OR+equivalent_classes_suffix+like+%27rdf%27+OR+equivalent_classes_suffix+like+%27syntax%27+OR+equivalent_classes_suffix+like+%27ns%23type%2C%27+OR+equivalent_classes_suffix+like+%27http%3A%2F%2Fdbpedia.org%2Fontology%2F%27+OR+equivalent_classes_suffix+like+%